In [1]:
import pandas as pd

In [2]:
category = pd.read_csv("wiki-topcats-categories.txt", header = None, delimiter = "/n", names=["nodes"])

/Users/dzidziquist/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
category.head()

,nodes
0,Category:Buprestoidea; 301 302 303 304 305 306...
1,Category:People_from_Worcester; 1056 1057 1058...
2,Category:Skin_conditions_resulting_from_physic...
3,Category:Visual_kei_bands; 1297 1300 1311 1312...
4,Category:Japanese_rock_music_groups; 1297 1300...


In [4]:
category["Name"]=category.apply(lambda x: x["nodes"].split(";")[0].split(":")[1], axis=1)

In [5]:
category

,nodes,Name
0,Category:Buprestoidea; 301 302 303 304 305 306...,Buprestoidea
1,Category:People_from_Worcester; 1056 1057 1058...,People_from_Worcester
2,Category:Skin_conditions_resulting_from_physic...,Skin_conditions_resulting_from_physical_factors
3,Category:Visual_kei_bands; 1297 1300 1311 1312...,Visual_kei_bands
4,Category:Japanese_rock_music_groups; 1297 1300...,Japanese_rock_music_groups
5,Category:Laboulbeniomycetes; 1941 1942 1943 19...,Laboulbeniomycetes
6,Category:Eye_anatomy; 2168 2702 2704 2705 2706...,Eye_anatomy
7,Category:Nervous_system; 951 1234 1806 2270 23...,Nervous_system
8,Category:Head_and_neck; 237 1277 1648 2298 272...,Head_and_neck
9,Category:Arteries_of_the_head_and_neck; 2726 2...,Arteries_of_the_head_and_neck


In [6]:
category["nodes"]= category.apply(lambda x: x["nodes"].split(";")[1].split(),axis=1)

In [7]:
category= category[["Name","nodes"]]

In [8]:
cat = category.loc[category.apply(lambda x: len(x["nodes"]), axis = 1) > 3500]

In [9]:
c = len(cat)

In [10]:
cat

,Name,nodes
868,English_footballers,"[22860, 28411, 28961, 28979, 29264, 29573, 295..."
869,The_Football_League_players,"[14003, 23536, 27109, 27348, 27459, 27989, 280..."
876,Association_football_forwards,"[26876, 26877, 26879, 26887, 26892, 26904, 269..."
898,Association_football_goalkeepers,"[26900, 26909, 26917, 26960, 26966, 26984, 270..."
900,Association_football_midfielders,"[14003, 15291, 23536, 26880, 26882, 26885, 268..."
949,Association_football_defenders,"[15217, 22860, 26873, 26878, 26881, 26898, 269..."
1047,Living_people,"[52, 62, 64, 66, 70, 74, 79, 96, 103, 104, 105..."
1881,Year_of_birth_unknown,"[3335, 10527, 15076, 16310, 22286, 23468, 2346..."
2791,Harvard_University_alumni,"[77, 1013, 1271, 1663, 1779, 1843, 2212, 3193,..."
4160,Major_League_Baseball_pitchers,"[79, 24213, 33054, 37167, 53973, 63107, 69823,..."


In [19]:
import pickle 

In [20]:
with open("Graph", "rb") as f:
    G = pickle.load(f)

In [21]:
G

{52: [401135, 1069112, 1163551],
 62: [12162,
  167659,
  279122,
  1089199,
  1354553,
  1400636,
  1403619,
  1537692,
  1544420],
 64: [64873],
 66: [279122, 1163290],
 74: [279122],
 95: [1179478, 1185516, 1185519, 1230865],
 103: [107],
 104: [107, 1174251, 1174302],
 105: [1174967],
 107: [104, 142167, 657930, 1013995, 1061780, 1174302, 1179210, 1181401],
 108: [104, 107, 1059989, 1062426, 1161925, 1169534],
 112: [107],
 113: [279122, 1185127],
 122: [1174967],
 126: [279122],
 133: [279122, 825464],
 134: [541222, 1060414, 1061485, 1062226, 1163610, 1163783, 1265505],
 137: [136, 1163286, 1163712, 1224802],
 153: [154,
  260842,
  725249,
  744272,
  1064807,
  1065282,
  1065320,
  1066493,
  1085943,
  1125916,
  1130618,
  1161083,
  1162104,
  1162238,
  1162816,
  1532744],
 154: [153, 279122, 673672, 1057077, 1063928, 1067637, 1175075],
 155: [279122, 1539938],
 156: [279122, 785921],
 158: [279122, 1262459, 1400635, 1403118],
 160: [598053, 1108161],
 163: [896828],
 166

In [22]:
import networkx as nx 

In [23]:
graph = nx.Graph(G)

In [24]:
#input category 
C0 = "English_footballers"
C1 = "Association_football_forwards"

In [ ]:
cate = []
for i in range(c-1):
    

In [ ]:
#distances between 2 categories 
nodeC0 = list(cat.loc[cat["Name"] == C0]["nodes"])[0]
nodeC1 = list(cat.loc[cat["Name"] == C1]["nodes"])[0]
shortestpath = []
for i in nodeC0:
    for j in nodeC1:
        try:
            length = nx.dijkstra_path_length(graph, int(i), int(j))
            shortestpath.append(length)
        except:
            continue

In [53]:
def dijkstra(G,a):
    visited=[a]
    d={}
    for i in G.nodes:
        d[i]=float('inf')
    d[a]=0
    #updating neighbors of a
    for i in list(G.neighbors(a)):
            d[i]=1
                
    while visited != G.nodes:
        nv = dict((k,d[k]) for k in set(G.nodes).difference(visited))
        v = min(nv, key=nv.get)
        print(v)
        visited.append(v)
        
        for i in list(G.neighbors(v)):
            if d[v]+1 <d[i]:
                d[i] = d[v]+1
    return d

In [54]:
dijkstra(graph, 62)

12162
1089199
167659
279122
1354553
1400636
1403619
1537692
1544420
1013825
1048610
1048628
66
1048642
1048645
74
1048663
1048674
1048675
1048683
113
1048689
126
1048706
133
1048709
1048710
1048715
1048722
1048723
154
155
156
158
1048731
1048733
1048735
167
1048743
1048771
1048777
209
1048793
1048795
1048796
1048798
1048800
1048808
1048821
1048828
256
1048835
1048836
1048837
1048844
1048850
1048853
1048861
1048879
1048881
1048887
1048888
1048902
1048915
1048918
1048922
1048924
1048932
1048955
1048962
1048970
1048989
1048993
1048995
1049002
1049008
1049016
1049027
1049036
1049048
1049054
1049062
1049076
1049084
1049087
1049098
1049103
1049108
1049112
1049145
1049146
1049155
1049156
1049158
1049161
1049193
1049209
1049218
1049220
1049222
1049239
1049240
1049244
1049250
1049261
1049277
1049340
1049341
1049343
1049344
1049346
1049356
1049357
1049358
1049361
1049365
1049382
1049383
1049384
1049385
1049386
1049387
1049388
1049393
1049399
1049400
1049402
1049406
1049408
1049419
1049420
847
10

1056993
1057006
1057020
1057022
1057024
1057031
1057032
1057040
1057041
1057046
1057054
1057057
1057061
1057063
1057074
1057076
1057081
1057085
1057093
1057094
1057096
1057097
1057101
1057108
1057114
1057118
1057122
1057125
1057131
1057138
1057155
1057158
1057161
1057173
1057174
1057175
1057177
1057185
1057189
8614
1057192
1057197
1057198
1057211
1057212
1057220
1057225
1057229
1057239
1057250
1057280
1057297
1057305
1057315
1057321
1057325
1057330
1057331
1057335
1057338
1057345
1057353
1057357
1057359
1057367
1057375
1057378
1057379
1057380
1057384
1057387
1057393
1057394
1057396
1057404
1057410
1057412
1057419
1057420
1057423
1057430
1057432
1057433
1057435
1057436
1057439
1057440
1057441
1057462
1057465
1057468
1057469
1057472
1057473
1057478
1057486
1057487
1057506
1057509
1057513
1057529
1057534
1057535
1057538
1057544
1057546
8971
1057553
8978
1057561
1057569
1057574
1057579
1057587
1057591
1057593
1057598
1057602
1057608
1057613
1057624
1057631
1057633
1057638
1057646
1057648
1

KeyboardInterrupt: 

In [49]:
d1={1:2,3:4,5:6}

In [50]:
d1[1,3]

KeyError: (1, 3)

In [48]:
min(d1, key=d1.get)

1